<a href="https://colab.research.google.com/github/jinwu99/Text-mining-and-Sentiment-analysis/blob/main/Crawling_from_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import googleapiclient.discovery

# API key
API_KEY = ''

# YouTube Data API client
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=API_KEY)

In [2]:
# 각 영상의 댓글 크롤링
# series : 시리즈물인 경우 넘버링도 함께 기록
def get_comments_exclude_host(video_id, channel_host_name, series):
    author_comments = []
    nextPageToken = None

    while True:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText',
            maxResults=100,
            pageToken=nextPageToken
        ).execute()

        for item in response['items']:
            comment_author = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
            comment_text = item['snippet']['topLevelComment']['snippet']['textDisplay']

            # 채널주인장의 댓글은 제외
            if comment_author != channel_host_name:
                author_comments.append([series,comment_author,comment_text])

            # 대댓글들이 있는 경우도 함꼐 저장하기위해
            top_level_comment_id = item['snippet']['topLevelComment']['id']
            replies_response = youtube.comments().list(
                part='snippet',
                parentId=top_level_comment_id,
                maxResults=100,
            ).execute()

            for reply_item in replies_response['items']:
                reply_author = reply_item['snippet']['authorDisplayName']
                reply_text = reply_item['snippet']['textDisplay']

                # 대댓글도 채널주인장의 댓글은 제외
                if reply_author != channel_host_name:
                    author_comments.append([series,reply_author, reply_text])

        nextPageToken = response.get('nextPageToken')

        if not nextPageToken:
            break

    return author_comments

사학도TV 남한산성 시리즈와 황현필 한국사 크롤링

In [ ]:
CHANNEL_HOST_NAME = '사학도TV'
VIDEO_ID = ['qCLuH_jB8-E','2vq0-x-b5Kc','8hh18YYY73E','iOgX-G7pAss',
            'GXkcghbfo2U','dQaPz6Azpz0','FV13AjxQnKs','-Qdr2KAYjiM','BvxUrx5DP6w']
youtube_review_list = []
series = 1
for video_id in VIDEO_ID:
    youtube_review_list += get_comments_exclude_host(video_id, CHANNEL_HOST_NAME, str(series))
    print(len(youtube_review_list))
    series += 1

CHANNEL_HOST_NAME = '황현필 한국사'
VIDEO_ID = 'bPSrXPOA1EQ'
youtube_review_list += get_comments_exclude_host(VIDEO_ID, CHANNEL_HOST_NAME, 'Hwang')

In [ ]:
df = pd.DataFrame(review_list, columns=["video", "user", "review"])
df.to_csv(f"Youtube_review.csv",index=False, encoding='utf-8-sig')